# House Prices - Advanced Regression Techniques

## Deva Kulkarni, Jared Dec, Marc Semonick, Trevor Johnson

## December 2021

<br>

The following report is based on the Kaggle competition: ["House Prices - Advanced Regression Techniques"](https://www.kaggle.com/c/house-prices-advanced-regression-techniques).  In the competition, participants are challenged to predict housing prices in Ames, Iowa based on a dataset complied by [James DeCock](https://http://jse.amstat.org/v19n3/decock.pdf).  The dataset contiains detailed data houses sold in Ames, Iowa between the years 2006 and 2010.  There are up to 80 data points per house, including expected hard data such as square feet for each floor and lot size, as well as some more subjective data like quality and condition.  The **inference problem** that we will attempt to solve in this report is: 

<p style="text-align: center;">Given these data points for a house, predict the sale price of that house.</p>

To guide the reader through our dolution of this inference problem, we will divide our approach into three main sections: Exploratory data analysis, feature engineering and model exploration, and our final predictive model.  The final deliverables will be this notebook containing all relevant code and descriptions, as well as a .csv output of our final predictions as required by the Kaggle competition.

# Setups

Import modules

In [2]:
import os 
import re

# data manipulation/viz
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

# modeling setups
from patsy import dmatrices
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.preprocessing import StandardScaler

# linear modeling
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
import statsmodels.api as sm
from statsmodels.formula.api import glm 

# tree modeling
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

# other
from sklearn.decomposition import PCA

# turn off the df['col'] = x assignment warning
#pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
# optional directory set-up
train = pd.read_csv("../../housing_data/train.csv")
test = pd.read_csv("../../housing_data/test.csv")
sample = pd.read_csv("../../housing_data/sample_submission.csv")